In [ ]:
from depmapomics.config import *
from depmapomics import tracker
from depmapomics import upload
from depmapomics import mutations
from genepy.utils import helper as h

In [ ]:
wgs_mat = mutations.generateGermlineMatrix(refworkspace=WGSWORKSPACE,
        vcfdir=WGSVCFDIR,
        savedir="output/" + SAMPLESETNAME + "/",
        filename="binary_mutguides_wgs.tsv.gz",
        bed_location=GUIDESBED,)

In [ ]:
import gumbo_client

client = gumbo_client.Client(username="szhang")

In [ ]:
pr_table = client.get("OmicsProfile")

In [ ]:
pr_table = client.get("omics_profile")

In [ ]:
import gumbo_client

client = gumbo_client.Client(username="szhang")
pr_table = client.get("omics_profile")

In [ ]:
pr_table.columns